# Read Data

In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
'''
This methods accepts:
    data - a list of words
    labels - a list of tags (as #s) corresponding to data (defaults to none for test data)
    tag_delim - the tag corresponding to -DOCSTART- that we split the tags on
    word_delim - the word to split the sentences on
Returns:
    sentences: a list of lists. Each sublist holds the words of a given sentence
    sentence_labels: a list of lists of the corresponding tags (as #s) to the sentences
    
NOTE: This is called within read_data
'''

def get_sentences(data, labels = None, tag_delim = 0, word_delim = '-DOCSTART-'):
    sentences = []
    for x, y in itertools.groupby(data, lambda z: z == word_delim):
        if x: sentences.append([])
        sentences[-1].extend(y)
    if labels is None:
        return sentences
    else:
        sentence_labels = []
        for x, y in itertools.groupby(labels, lambda z: z == tag_delim):
            if x: sentence_labels.append([])
            sentence_labels[-1].extend(y)
        return sentences, sentence_labels    

In [3]:
'''
This method accepts:
    data_file and label_file (optional) - file names for words and corresponding tags
Returns:
    sentences - a list of sentences, where each sentence is represented as a list of words and begins with -DOCSTART-
    sentences_tags - a list of lists of tags corresponding to the sentences, where tags are represented as integers
    tag_list - a list of the unique tags. The index of each tag is what we replace all tags with. 
            Later, we will use this list to convert number tags back to actual tags:
            tags = [tag_list[x] for x in tags]
            
'''
def read_data(data_file, label_file = None):
    data = pd.read_csv(data_file)
    del data['id'] 
    if label_file is None:
        return get_sentences(list(data['word']))
    else:
        labels = pd.read_csv(label_file)
        del labels['id']
        
        # convert labels to numbers and store the conversion from # back to tag in a dictionary tag_list
        labels['tag'] = labels['tag'].astype('category')
        tag_list = list(labels['tag'].cat.categories)
        docstart_tag = tag_list.index('O') # get # corresponding to tag 'O'
        labels = np.array(labels['tag'].cat.codes)

        sentences, sentences_tags = get_sentences(list(data['word']), labels, tag_delim = docstart_tag)
        return sentences, sentences_tags, tag_list

In [4]:
# This is how we call read_data with all the files:
# DIEGO: Make sure that in read_data. dev and train  have the same tag_list distribution. They can't have different tag numbers.
dev_x, dev_y, tag_list = read_data('data/dev_x.csv', 'data/dev_y.csv')
train_x, train_y, tag_list = read_data('data/train_x.csv', 'data/train_y.csv')
test_x = read_data('data/test_x.csv')

In [ ]:
train_x[2]

# Dictionary

In [5]:
class Dictionary:

    def __init__(self):
        self.emissions = None
        self.transitions = None
    
    '''
    return all emissions (for all words,tags)
    this is structured as a dictionary where each key is a word and it's value is a numpy array with a value for each tag
    To get e(word|tag), you do emissions[word][tag]
    '''
    def getEmissions(self):
        return self.emissions

    '''returns emissions for a given word as a numpy array where each value corresponds to a tag => e(x|y) for all y values'''
    def getEmissionsByWord(self, word):
        return self.emissions[word]

    '''
    returns all transitions(for all states)
    this is structured as a dictionary where each key is a previous state and it's value is a numpy array with a value for each tag
    To get q(tag|prev_state), you do transitions[prev_state][tag]
    Prev_state is the previous tag in the bigram case, and previous 2 tags joined with ',' in trigram case
    '''
    def getTransitions(self):
        return self.transitions
   
    '''
    same as getTransitions but returns all transitions for a given state.
    '''
    def getTransitionsByState(self, prev):
        return self.transitions[prev]

    '''
    prev = previous state as string
    curr = current state (tag)
    returns q(curr|prev)
    '''
    def getTransition(self, prev, curr):
        return self.transitions[prev][curr]
    
    '''
    This method accepts:
        sentences - list of sentences, each represented as a list of words
        sentences_tags - tags (as #s) corresponding to the words in sentences
        tag_list - list of unique tags
        n_prev - # of previous tags to consider. 2 for trigram, 1 for bigram
    This calculates the transitions and emissions and stores them in the member variables
    '''
    def process(self, sentences, sentences_tags, tag_list, n_prev):
        num_tags = len(tag_list)
        self.__calculate_transitions(sentences_tags, num_tags, n_prev)
        self.__calculate_emissions(sentences, sentences_tags, num_tags)
        return
    
    '''
    Given the tags for each sentence and the number of unique tags,
    this method returns a dictionary where key = tag and value = tag's count 
    '''
    def __calculate_tag_counts(self, sentences_tags, num_tags):
        tag_counts = [0] * num_tags
        for sent_tags in sentences_tags:
            for tag in sent_tags:
                tag_counts[tag] +=1
        return tag_counts
    
    '''
    This method, called in process, calculates the emissions, 
    structured as a dictionary where each key is a word and it's value is a numpy array with a value for each tag.
    To get e(word|tag), you do e[word][tag]
    Given: list of sentences, corresponding tags, and number of unique tags
    Emissions are calculated as the log((count(word, tag) + 1) / (count(tag) + ?????)) - using add-1 smoothing
    DIEGO: ???? should be the size of dictionary of words.
    
    ****************
    QUESTION - If I'm starting all count(word,tag) at 1, do I also need to be doing that for count(tag)
        DIEGO: Yes. Otherwise you are providing inconsistency in the distribution. As I said, the counts should all start in zero
        and the Add-1 should only be done when computing the log to get the probability distribution. It's much easier to track
        the places where it's used and how it's used. You avoid adding 1 twice or none at all.
    ***** Fix ^^^
        DIEGO: FIXED
    
    '''
    def __calculate_emissions(self, sentences, sentences_tags, num_tags):
        e = {}
        # DIEGO: This can be included within the first loop which would save an iteration over the entire set of tags.
        tag_counts = self.__calculate_tag_counts(sentences_tags, num_tags) # get dictionary of tag counts
        
        # First go through all words/labels and count all (word,label) pairs
        for sent, sent_tags in itertools.izip(sentences, sentences_tags):
            for word, label in itertools.izip(sent, sent_tags):
                if not word in e:
                    e[word] = np.zeros(num_tags)
                e[word][label] +=1   

        dict_size = len(e)
                
        # Divide the counts(word,tag) by count(tag) and log it 
        for i in range(num_tags):
            for word, value in e.iteritems():
                 # We compute values using the log and also use add 1 smoothing
                e[word][i] = np.log( (e[word][i] + 1) / (tag_counts[i] + dict_size))

        # NOTE: STILL NEED TO TAKE CARE OF UNKNOWN WORDS BY DOING THE FREQUENCY THING!!!

        del tag_counts
        self.emissions = e
        return
    
    '''
    This method, called in process, calculates the transitions, 
    structured as a dictionary where each key is a previous state and it's value is a numpy array with a value for each tag.
    To get q(tag|prev_state), you do transitions[prev_state][tag]
    Prev_state is the previous tag in the bigram case, and previous 2 tags joined with ',' in trigram case
    Given: 
        sentences_tags - list of tags separated by sentence, 
        num_tags - number of unique tags
        n - number of previous states to use. 2 for trigram, 1 for bigrram
    Transitions are calculated as the log((count(prev_state, tag) + 1) / (count(prev_state) + num_tags)) - using add-1 smoothing
    '''
    def __calculate_transitions(self, sentences_tags, num_tags, n):
        q = {}
        for state in itertools.product(range(-1,num_tags), repeat = n):
            q[','.join(str(tag) for tag in state)] = np.zeros(num_tags)

        for sent_tags in sentences_tags:  # for each sentence's list of tags
            sent_tags = ([-1] * n) + sent_tags  #appending n '-1' tags to correspond to prior states/tags for the first word(s). 

            for i in range(len(sent_tags) - n):
                prior_state = ','.join(str(sent_tags[j]) for j in range(i, i + n)) # Create a string with n sequenced tags
                current_state = sent_tags[i + n]
                q[prior_state][current_state] += 1

        for prior_state, tag_list in q.iteritems():
            q[prior_state] = np.log( (tag_list + 1) / (np.sum(tag_list) + num_tags))
            
        self.transitions = q   
        return

# Example using the code above

In [ ]:
d = Dictionary()

In [ ]:
sentences = [['i', 'went','to','i'],['to','Diego'],['love', 'i']]
sentences_tags = [[1,0,2,1],[2,3],[4,1]]
num_tags = 5
tag_list = [0,1,2,3,4]
n_prev = 2 # 2 is for trigram. 1 is for bigram

d.process(sentences, sentences_tags, tag_list, n_prev)

In [ ]:
d.getEmissions()

In [ ]:
d.getEmissionsByWord('Diego')

In [ ]:
d.getTransitions()

In [ ]:
d.getTransition('1,0', 2)

# TODO

TODO: TAKE CARE OF UNKNOWN WORDS AND SMOOTHING 
(FIX VITERBI TO GET RID OF -1s)
Fix question that I wrote in calculate_emissions method

# VITERBI

In [ ]:
sentence = ['i', 'went','to','i']

In [6]:
#VITERBI
n = 2 # trigram
pis = [1] * num_tags
paths = [[-1]*n] * num_tags

for word in sentence:
    e = d.getEmissionsByWord(word)
    new_pis = [-float("inf")] * num_tags
    new_paths = [[]] * num_tags
    for tag in range(num_tags):
        e_pi = e[tag]
        for prev_pi, prev_path in itertools.izip(pis, paths):
            prev_state = ','.join(str(prev_path[j]) for j in range(-n, 0))
            q_pi = d.getTransition(prev_state, tag)
            pi = e_pi + prev_pi + q_pi
            if pi > new_pis[tag]:
                new_pis[tag] = pi
                new_paths[tag] = prev_path + [tag]
            
    pis = new_pis
    paths = new_paths


best_path = paths[np.argmax(np.array(pis))]

NameError: name 'num_tags' is not defined

In [ ]:
best_path

# Run with real data

In [7]:
d = Dictionary()
d.process(train_x, train_y, tag_list, n_prev = 2)

In [ ]:
d.getEmissions()

In [ ]:
d.getEmissionsByWord('the')

In [ ]:
tag_list[10] # the greates number in e for 'the' was in index 10, which is 'DT' --> makes sense

In [ ]:
d.getTransitions()

In [ ]:
d.getTransition('1,0', 2)

In [8]:
def viterbi(sentence, d, num_tags = 45, n = 2):

    pis = [1] * num_tags
    paths = [[-1]*n] * num_tags

    for word in sentence:
        e = d.getEmissionsByWord(word)
        new_pis = [-float("inf")] * num_tags
        new_paths = [[]] * num_tags
        for tag in range(num_tags):
            e_pi = e[tag]
            for prev_pi, prev_path in itertools.izip(pis, paths):
                prev_state = ','.join(str(prev_path[j]) for j in range(-n, 0))
                q_pi = d.getTransition(prev_state, tag)
                pi = e_pi + prev_pi + q_pi
                if pi > new_pis[tag]:
                    new_pis[tag] = pi
                    new_paths[tag] = prev_path + [tag]

        pis = new_pis
        paths = new_paths


    best_path = paths[np.argmax(np.array(pis))]
    return best_path[n:]

In [11]:
path = viterbi(train_x[0], d)

In [13]:
# checking current accuracy: 0.78 on train data -- not so great 
# need to fix the smoothing and unknown stuff
np.sum(path[i] == train_y[0][i] for i in range(len(path)))/float(len(path))

0.9375

In [ ]:
d.getTransitions().keys()

In [14]:
len(train_x)

1387

In [ ]:
dev_x[25]

In [ ]:
elem = dev_x[19]
path = viterbi(elem, d)
np.sum(path[i] == elem[i] for i in range(len(path)))/float(len(path))